In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

In [ ]:
(Dataset, _),(_, _) = tf.keras.datasets.cifar10.load_data()

170508288/170498071 [==============================] - 2s 0us/step


In [ ]:
Dataset = Dataset / 255

In [ ]:
loss_func = tf.keras.losses.BinaryCrossentropy()
disc_opt = tf.keras.optimizers.Adam(learning_rate= 0.0002)
gen_opt = tf.keras.optimizers.Adam(learning_rate= 0.0002)

latent_dim = 100

In [ ]:
batchsize = 32

In [ ]:
#discriminator
Discriminator = tf.keras.Sequential(
    [
     tf.keras.Input(shape= (32,32,3)),
     tf.keras.layers.Conv2D(32,kernel_size=5,strides=2,padding='same'),
     tf.keras.layers.LeakyReLU(0.2),
     tf.keras.layers.BatchNormalization(),

     tf.keras.layers.Conv2D(64,kernel_size=5,strides=2,padding='same'),
     tf.keras.layers.LeakyReLU(0.2),
     tf.keras.layers.BatchNormalization(),

     tf.keras.layers.Conv2D(128,kernel_size=5,strides=2,padding='same'),
     tf.keras.layers.LeakyReLU(0.2),
     tf.keras.layers.BatchNormalization(),

    #  tf.keras.layers.Conv2D(256,kernel_size=5,strides=2,padding='same'),
    #  tf.keras.layers.LeakyReLU(0.1),
    #  tf.keras.layers.BatchNormalization(),

     tf.keras.layers.Flatten(),
     tf.keras.layers.Dense(1,activation='sigmoid')
     
    ]
    
)

In [ ]:
#Generator
Generator = tf.keras.Sequential(
    [
     tf.keras.Input(shape=(latent_dim)),
     tf.keras.layers.Dense(8*8*128),
     tf.keras.layers.LeakyReLU(0.2),
     tf.keras.layers.BatchNormalization(),
     tf.keras.layers.Reshape(target_shape=(8,8,128)),

     tf.keras.layers.Conv2DTranspose(128,kernel_size=5,strides=1,padding='same'),   #output shape (8,8,128)
     tf.keras.layers.LeakyReLU(0.2),
     tf.keras.layers.BatchNormalization(),
     
     tf.keras.layers.Conv2DTranspose(64,kernel_size=5,strides=2,padding='same'),   #output shape (16,16,64)
     tf.keras.layers.LeakyReLU(0.2),
     tf.keras.layers.BatchNormalization(),  

     tf.keras.layers.Conv2DTranspose(3,kernel_size=5,strides=2,padding='same',activation='tanh') #output shape (32,32,3)
    ]
)

In [ ]:
Disc = Discriminator.load_weights(filepath='/content/drive/MyDrive/DCGANS M weights/Discriminator')
Gen = Generator.load_weights(filepath='/content/drive/MyDrive/DCGANS M weights/Generator')
Disc.assert_consumed()
Gen.assert_consumed()


In [ ]:
#Training Loop

for epoch in range(10):
  for index , real in enumerate(Dataset):
    random_latent_vactors = tf.random.normal(shape=(batchsize,latent_dim))
    fake_imgs = Generator(random_latent_vactors)
    actual_imgs = Dataset[np.random.randint(0,Dataset.shape[0] ,32)]

    if index%100 == 0:
      img = tf.keras.preprocessing.image.array_to_img(fake_imgs[0])
      img.save(f'/content/drive/MyDrive/sample images of saved/image{index}.png')
      Discriminator.save_weights(filepath='/content/drive/MyDrive/DCGANS M weights/Discriminator')
      Generator.save_weights(filepath='/content/drive/MyDrive/DCGANS M weights/Generator')

    # Training of Discriminator
    with tf.GradientTape() as disc_tape:
      fake_loss = loss_func(tf.zeros(shape = (batchsize,1)),Discriminator(fake_imgs))
      real_loss = loss_func(tf.ones(shape=(batchsize,1)),Discriminator(actual_imgs))
      total_disc_loss = (fake_loss + real_loss)/2

    grades = disc_tape.gradient(total_disc_loss,Discriminator.trainable_weights)
    disc_opt.apply_gradients(zip(grades,Discriminator.trainable_weights))
   
    print('index = ',index)
    print("Epoch = ",epoch)
    print('Discriminator ',total_disc_loss)

    # Training of Generator 
    with tf.GradientTape() as gen_tape:
      random_latent_vactors = tf.random.normal(shape=(batchsize,latent_dim))
      gen_fake = Generator(random_latent_vactors)
      output = Discriminator(gen_fake)
      total_gen_loss = loss_func(tf.ones(shape=(batchsize,1)),output)

    grades = gen_tape.gradient(total_gen_loss,Generator.trainable_weights)
    gen_opt.apply_gradients(zip(grades,Generator.trainable_weights))
    
    print('Genrator ',total_gen_loss)





Streaming output truncated to the last 5000 lines.
index =  36433
Epoch =  3
Discriminator  tf.Tensor(0.69639826, shape=(), dtype=float32)
Genrator  tf.Tensor(0.92404425, shape=(), dtype=float32)
index =  36434
Epoch =  3
Discriminator  tf.Tensor(0.6471937, shape=(), dtype=float32)
Genrator  tf.Tensor(0.81236166, shape=(), dtype=float32)
index =  36435
Epoch =  3
Discriminator  tf.Tensor(0.58064264, shape=(), dtype=float32)
Genrator  tf.Tensor(0.6772298, shape=(), dtype=float32)
index =  36436
Epoch =  3
Discriminator  tf.Tensor(0.59930277, shape=(), dtype=float32)
Genrator  tf.Tensor(0.72791636, shape=(), dtype=float32)
index =  36437
Epoch =  3
Discriminator  tf.Tensor(0.6498389, shape=(), dtype=float32)
Genrator  tf.Tensor(0.7894735, shape=(), dtype=float32)
index =  36438
Epoch =  3
Discriminator  tf.Tensor(0.63014644, shape=(), dtype=float32)
Genrator  tf.Tensor(0.67906994, shape=(), dtype=float32)
index =  36439
Epoch =  3
Discriminator  tf.Tensor(0.5936554, shape=(), dtype=float

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
